1. INITIALIZATION

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import os
from PIL import Image
import shutil
from sklearn.model_selection import train_test_split
from skimage.util import random_noise
import random
from albumentations import Compose, HorizontalFlip, Rotate, GaussianBlur, RandomBrightnessContrast, GaussNoise
from albumentations.core.composition import OneOf
from google.colab import drive
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


2. LOADING DATASET

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_path = '/content/drive/MyDrive/dataset_unique'

3. PREPOCESSING IMAGE

Since the dataset had previously undergone augmentation, we found that the augmentation was not evenly applied across all classes and, in some cases, was too extreme. As a result, we decided to temporarily remove the augmentation to ensure a more balanced and controlled approach before reapplying it in a more consistent manner.

In [ ]:
def delete_augmented_images(input_dir, prefix="aug"):
    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                if file_name.startswith(prefix):
                    file_path = os.path.join(class_path, file_name)
                    try:
                        os.remove(file_path)
                        print(f"Deleted augmented image: {file_path}")
                    except Exception as e:
                        print(f"Error deleting file {file_path}: {e}")

In [ ]:
delete_augmented_images(dataset_path)

Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_9981.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_30.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_550.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_6527.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_7268.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_4628.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_3221.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_8947.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_8725.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_7265.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahma/aug_0_9725.jpg
Deleted augmented image: /content/drive/MyDrive/dataset_unique/brahm

In [ ]:
# Loop through the classes and count the number of files in each class folder
class_counts = {}

# Iterate over the folders (each folder is a class)
for class_name in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path, class_name)

    # Only count directories (classes)
    if os.path.isdir(class_folder):
        # Count files in the class folder
        file_count = len([f for f in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, f))])
        class_counts[class_name] = file_count

# Display the count for each class
for class_name, count in class_counts.items():
    print(f"Class '{class_name}': {count} files")

Class 'siwa': 797 files
Class 'wisnu': 824 files
Class 'brahma': 620 files
Class 'durga': 619 files
Class 'gupolo': 762 files
Class 'ganesa': 619 files
Class 'nandhi': 617 files
Class 'agastya': 620 files
Class 'surya': 615 files
Class 'candra': 482 files


Since we observed that the data distribution was still imbalanced, we decided to perform re-augmentation on the classes with fewer than 620 samples. This step was taken to ensure a more even distribution of samples across all classes, improving the overall balance of the dataset.

In [ ]:


def apply_motion_blur(image, kernel_size=3):
    # Mengurangi kernel size untuk mengurangi tingkat keburaman
    if kernel_size < 3:
        kernel_size = 3

    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)
    kernel = kernel / kernel_size
    blurred_image = cv2.filter2D(image, -1, kernel)

    # Meningkatkan ketajaman gambar setelah blur dengan filter sharpen
    sharpen_kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    sharpened_image = cv2.filter2D(blurred_image, -1, sharpen_kernel)

    return sharpened_image

def add_gaussian_noise(image, variance=0.0005):
    mean = 0
    stddev = np.sqrt(variance)

    # Menghasilkan Gaussian noise
    gaussian_noise = np.random.normal(mean, stddev, image.shape).astype(np.float32)

    # Menambahkan noise pada gambar asli
    noisy_image = image.astype(np.float32) / 255.0 + gaussian_noise
    noisy_image = np.clip(noisy_image, 0, 1.0)  # Rentang nilai 0-1
    noisy_image = (noisy_image * 255).astype(np.uint8)  # Konversi ke rentang 0-255 dan tipe uint8

    return noisy_image


In [ ]:
def augment_images(input_dir, min_images=620, target_size=(224, 224)):
    datagen = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=5,  # Mengurangi rotasi maksimal menjadi 5 derajat
        width_shift_range=0.1,  # Mengurangi pergeseran horizontal
        height_shift_range=0.1,  # Mengurangi pergeseran vertikal
        shear_range=0.1,  # Mengurangi shear effect
        zoom_range=0.1,  # Mengurangi zoom effect
        horizontal_flip=True,  # Tetap menggunakan flip horizontal
        fill_mode='nearest'  # Menjaga area kosong setelah transformasi
    )

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if os.path.isdir(class_path):
            images = os.listdir(class_path)
            num_images = len(images)
            if num_images < min_images:
                num_to_generate = min_images - num_images
                print(f"Augmenting {num_to_generate} images for class {class_name}")

                for i in range(num_to_generate):
                    img_name = images[i % num_images]
                    img_path = os.path.join(class_path, img_name)
                    try:
                        img = cv2.imread(img_path)
                        if img is None:
                            print(f"Failed to load image: {img_path}")
                            continue

                        img = cv2.resize(img, target_size)

                        # Efek augmentasi tambahan secara acak
                        if random.choice([True, False]):
                            img = apply_motion_blur(img)
                        if random.choice([True, False]):
                            img = add_gaussian_noise(img)

                        x = keras.preprocessing.image.img_to_array(img)
                        x = x.reshape((1,) + x.shape)

                        for batch in datagen.flow(x, batch_size=1, save_to_dir=class_path, save_prefix='aug', save_format='jpg'):
                            break
                    except Exception as e:
                        print(f"Error augmenting image {img_path}: {e}")


In [ ]:
augment_images(dataset_path)

Augmenting 1 images for class durga
Augmenting 1 images for class ganesa
Augmenting 3 images for class nandhi
Augmenting 5 images for class surya
Augmenting 138 images for class candra


We resized the images to 224x224 pixels to standardize the input size, ensuring consistency across all images in the dataset. This resizing is essential for training the model, as it allows the network to process the images uniformly.

In [ ]:
def resize_images(input_dir, output_dir, target_size=(224, 224)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)
        if not os.path.exists(output_class_path):
            os.makedirs(output_class_path)

        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    with Image.open(img_path) as img:
                        img_resized = img.resize(target_size)
                        img_resized.save(os.path.join(output_class_path, img_name))
                except Exception as e:
                    print(f"Error resizing image {img_path}: {e}")

In [4]:
resized_dataset_path = '/content/drive/MyDrive/resized_dataset_unique'

In [ ]:
resize_images(dataset_path, resized_dataset_path)

We split the dataset into three parts: 60% for training, 20% for validation, and 20% for testing. This division ensures that the model has enough data to learn from during training, while also providing separate sets for evaluating its performance and fine-tuning hyperparameters.

In [ ]:
def split_dataset(input_dir, output_dir, train_ratio=0.6, val_ratio=0.2, test_ratio=0.2):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if os.path.isdir(class_path):
            images = os.listdir(class_path)
            train, temp = train_test_split(images, train_size=train_ratio, random_state=42)
            val, test = train_test_split(temp, test_size=test_ratio / (val_ratio + test_ratio), random_state=42)

            for split_name, split_data in zip(['train', 'val', 'test'], [train, val, test]):
                split_path = os.path.join(output_dir, split_name, class_name)
                if not os.path.exists(split_path):
                    os.makedirs(split_path)
                for img_name in split_data:
                    shutil.copy(os.path.join(class_path, img_name), os.path.join(split_path, img_name))

split_dataset(resized_dataset_path, '/content/drive/MyDrive/split_dataset_unique')

In [5]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
val_test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/split_dataset_unique/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_test_datagen.flow_from_directory(
    '/content/drive/MyDrive/split_dataset_unique/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    '//content/drive/MyDrive/split_dataset_unique/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 4689 images belonging to 10 classes.
Found 1698 images belonging to 10 classes.
Found 1674 images belonging to 10 classes.


In [7]:
from collections import Counter
train_labels = train_generator.classes
print("Distribusi akhir train:", Counter(train_labels))
print("Distribusi akhir validation:", Counter(train_labels))
print("Distribusi akhir test:", Counter(train_labels))

Distribusi akhir train: Counter({6: 634, 9: 494, 7: 478, 4: 468, 5: 457, 1: 452, 2: 442, 0: 433, 3: 416, 8: 415})
Distribusi akhir validation: Counter({6: 634, 9: 494, 7: 478, 4: 468, 5: 457, 1: 452, 2: 442, 0: 433, 3: 416, 8: 415})
Distribusi akhir test: Counter({6: 634, 9: 494, 7: 478, 4: 468, 5: 457, 1: 452, 2: 442, 0: 433, 3: 416, 8: 415})
